In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset-1000.csv')
df.head()

,Name,Age,Region,Languages,Languages-2,Subject,Preferred_Time,Preferred_Time-2,Preferred_Days,Preferred_Days-2,Preferred_Days-3,Preferred_Days-4,Preferred_Days-5,Preferred_Days-6,Preferred_Days-7,Mode,Mode-2,ExperiencePhysics,ExperienceChemistry,ExperienceMaths
0,Trunks Vegeta,58,Andaman and Nicobar Islands,Assamese,"Santali, Kashmiri","Chemistry, Maths",Evening,"Morning, Afternoon, Evening",Wednesday,"Tuesday, Monday","Monday, Wednesday, Saturday","Monday, Tuesday, Wednesday, Sunday","Monday, Tuesday, Thursday, Friday, Saturday","Monday, Tuesday, Wednesday, Friday, Saturday, ...","Monday, Tuesday, Wednesday, Thursday, Friday, ...",doubt solving,"Video lectures,live lectures",0,6,38
1,Black Frieza,42,Maharashtra,Odia,"Telugu, Assamese",Chemistry,Night,"Afternoon, Night",Sunday,"Thursday, Saturday","Monday, Wednesday, Friday","Monday, Friday, Saturday, Sunday","Monday, Tuesday, Thursday, Friday, Saturday","Monday, Wednesday, Thursday, Friday, Saturday,...","Monday, Tuesday, Wednesday, Thursday, Friday, ...",doubt solving,"Video lectures, one-on-one, live lectures",0,26,0
2,Ayush Singh,68,Meghalaya,Tulu,"Santali, Mewari",Maths,Afternoon,"Morning, Afternoon",Wednesday,"Thursday, Wednesday","Tuesday, Wednesday, Saturday","Monday, Tuesday, Wednesday, Friday","Monday, Tuesday, Wednesday, Friday, Saturday","Monday, Wednesday, Thursday, Friday, Saturday,...","Monday, Tuesday, Wednesday, Thursday, Friday, ...",one-on-one,"Video lectures, one-on-one, doubt solving",0,0,1
3,Armin Arlert,49,Punjab,Laman/Lambadi,"Sanskrit, Bengali",Maths,Morning,"Afternoon, Night",Tuesday,"Friday, Wednesday","Friday, Saturday, Sunday","Tuesday, Thursday, Friday, Sunday","Monday, Tuesday, Wednesday, Friday, Saturday","Tuesday, Wednesday, Thursday, Friday, Saturday...","Monday, Tuesday, Wednesday, Thursday, Friday, ...",Video lectures,"Video lectures,live lectures",0,0,42
4,Eren Yeager,66,Telangana,Malvi,"Maithili, Sanskrit","Physics, Maths",Night,"Morning, Afternoon, Evening",Sunday,"Wednesday, Friday","Monday, Tuesday, Wednesday","Tuesday, Friday, Saturday, Sunday","Monday, Wednesday, Thursday, Saturday, Sunday","Monday, Tuesday, Wednesday, Thursday, Friday, ...","Monday, Tuesday, Wednesday, Thursday, Friday, ...",one-on-one,"Video lectures, one-on-one, doubt solving, liv...",29,0,45


In [3]:
temp = df[[ 'Name' ,'Languages' ,"ExperiencePhysics",	"ExperienceChemistry"	, "ExperienceMaths" , "Subject"	 , "Preferred_Time" , "Region" , 'Mode'	]]

In [4]:
temp.head()

,Name,Languages,ExperiencePhysics,ExperienceChemistry,ExperienceMaths,Subject,Preferred_Time,Region,Mode
0,Trunks Vegeta,Assamese,0,6,38,"Chemistry, Maths",Evening,Andaman and Nicobar Islands,doubt solving
1,Black Frieza,Odia,0,26,0,Chemistry,Night,Maharashtra,doubt solving
2,Ayush Singh,Tulu,0,0,1,Maths,Afternoon,Meghalaya,one-on-one
3,Armin Arlert,Laman/Lambadi,0,0,42,Maths,Morning,Punjab,Video lectures
4,Eren Yeager,Malvi,29,0,45,"Physics, Maths",Night,Telangana,one-on-one


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


# Step 2: Feature Engineering (Example: Using 'languages', 'TeachingStyle', and 'ResearchInterests' text features)
# Vectorize each text feature separately using TF-IDF
tfidf_vectorizer_languages = TfidfVectorizer()
tfidf_matrix_languages = tfidf_vectorizer_languages.fit_transform(temp['Languages'].fillna(''))

tfidf_vectorizer_teaching_style = TfidfVectorizer()
tfidf_matrix_subject = tfidf_vectorizer_teaching_style.fit_transform(temp['Subject'].fillna(''))

tfidf_vectorizer_research_interests = TfidfVectorizer()
tfidf_matrix_time = tfidf_vectorizer_research_interests.fit_transform(temp['Preferred_Time'].fillna(''))

tfidf_vectorizer_research_interests = TfidfVectorizer()
tfidf_matrix_mode = tfidf_vectorizer_research_interests.fit_transform(temp['Mode'].fillna(''))

# Standardize numerical features if needed (as in the previous example)
numerical_features = temp[['ExperiencePhysics',	'ExperienceChemistry',	'ExperienceMaths']]
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features.fillna(0))  # Fill NaN values with 0 or handle them appropriately

# Combine the text and numerical features into a single feature matrix
import scipy.sparse as sp
feature_matrices = [tfidf_matrix_languages , tfidf_matrix_subject ,tfidf_matrix_time , tfidf_matrix_mode ,  numerical_features_scaled ]
tfidf_matrix_combined = sp.hstack(feature_matrices, format='csr')

# Step 4: Similarity Calculation (Example: Using Cosine Similarity)
# Calculate cosine similarity between teachers based on their combined features
cosine_sim = cosine_similarity(tfidf_matrix_combined, tfidf_matrix_combined)


# print(cosine_sim.shape)
# Step 5: Recommendation Generation (Same as previous examples)
# For a given student's preferences, recommend teachers with high similarity

cols = ['Languages' , 'Subject' ,'Preferred_Time' , 'Region' , 'Mode' ]

def recommend_teachers(student_preferences):
    teacher_indices = []

    for  feat , preference in zip(cols , student_preferences):
        teacher_idx = temp.index[temp[feat].str.contains(preference)].tolist()
        teacher_indices.extend(teacher_idx)

        # print(teacher_idx)


    mlikely = cosine_sim[teacher_idx[0]]
    teacher_indices = list(set(teacher_indices))  # Remove duplicates
    teacher_scores = [(temp.iloc[teacher_idx, 0], mlikely[teacher_idx]) for teacher_idx in teacher_indices]
    teacher_scores = sorted(teacher_scores, key=lambda x: x[1], reverse=True)
    print(teacher_scores)
    return teacher_scores

# Step 7: User Interface (Example: Taking student preferences as input)
student_preferences = ['Maharashtra', "Assamese", 'Physics' , 'Evening' , 'doubt solving']
recommended_teachers = recommend_teachers(student_preferences)


# Display recommended teachers
for teacher, score in recommended_teachers:
    print(f'Teacher ID: {teacher}, Similarity Score: {score}')

# You can add additional filtering and ranking logic in Step 6

[('Trunks Vegeta', 0.9999999999999998), ('Kokushibo', 0.8884652389686802), ('Anil Chhangani', 0.8474300198377009), ('Son Goku', 0.8296189673863799), ('Itadori Kyuji', 0.8277965606364661), ('Muzan Kibutsuji', 0.8277965606364661), ('Tengen Uzui', 0.8117225635731213), ('War Hammer Titan', 0.7946667806613747), ('Eld Jinn', 0.7885347122758894), ('Muichiro Tokito', 0.7769960697667861), ('Saiyan Cabba', 0.7622910104484809), ('Komalika Acharya', 0.7458493405814314), ('Beast Titan', 0.7434310281013868), ('Black Frieza', 0.7319330071167558), ('Takemichi Hanagaki', 0.7232238468808294), ('Prem  Acharya', 0.7132332330509171), ('Angel Whis', 0.705218075414374), ('Female Titan', 0.7048619697711385), ('Yoriichi Tsugikuni', 0.704006086877998), ('Piush Paul', 0.7010691018640829), ('Megumi Fushiguro', 0.6989257437287744), ('Muzan Kibutsuji', 0.6978482684916396), ('Ken Ryuguji', 0.6947544638921423), ('Hange Zoë', 0.6943268055673553), ('Armored Titan', 0.6939745789117573), ('Master Roshi', 0.69150846400311